In [1]:
!./start_hadoop_spark.sh

Starting HDFS...


Starting namenodes on [okeanos-master]
Starting datanodes
Starting secondary namenodes [okeanos-master]
Starting YARN...
Starting resourcemanager
Starting nodemanagers
Starting Spark...
starting org.apache.spark.deploy.history.HistoryServer, logging to /home/user/opt/spark/logs/spark-user-org.apache.spark.deploy.history.HistoryServer-1-okeanos-master.out


In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName('Project') \
        .config("spark.master", "yarn") \
        .config("spark.executor.instances", "4") \
        .config("spark.executor.cores", "1") \
        .config("spark.executor.memory", "1g") \
        .getOrCreate()
spark


23/12/23 19:20:45 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


## Ζητούμενο 2

In [6]:
from pyspark.sql.functions import col, to_date, to_timestamp

# columns_to_select = ["Date Rptd", "DATE OCC", "Vict Age","LAT","LON"]

# Read the CSV file into a DataFrame
df1 = spark.read.csv('hdfs://okeanos-master:54310/user/project/Crime_Data_from_2010_to_2019.csv', header=True, inferSchema=True)

df1 = df1.withColumn("Date Rptd", to_date(col("Date Rptd"), 'MM/dd/yyyy hh:mm:ss a'))
df1 = df1.withColumn("DATE OCC", to_date(col("DATE OCC"), 'MM/dd/yyyy hh:mm:ss a'))

# Read the CSV file into a DataFrame
df2 = spark.read.csv('hdfs://okeanos-master:54310/user/project/Crime_Data_from_2020_to_Present.csv', header=True, inferSchema=True)

df2 = df2.withColumn("Date Rptd", to_date(col("Date Rptd"), 'MM/dd/yyyy hh:mm:ss a'))
df2 = df2.withColumn("DATE OCC", to_date(col("DATE OCC"), 'MM/dd/yyyy hh:mm:ss a'))

df = df1.union(df2)

df.show()

+---------+----------+----------+--------+-----+---------+-----------+--------+------+--------------------+--------------+--------+--------+------------+---------+--------------------+--------------+--------------------+------+------------+--------+--------+--------+--------+--------------------+--------------------+-------+---------+
|    DR_NO| Date Rptd|  DATE OCC|TIME OCC|AREA |AREA NAME|Rpt Dist No|Part 1-2|Crm Cd|         Crm Cd Desc|       Mocodes|Vict Age|Vict Sex|Vict Descent|Premis Cd|         Premis Desc|Weapon Used Cd|         Weapon Desc|Status| Status Desc|Crm Cd 1|Crm Cd 2|Crm Cd 3|Crm Cd 4|            LOCATION|        Cross Street|    LAT|      LON|
+---------+----------+----------+--------+-----+---------+-----------+--------+------+--------------------+--------------+--------+--------+------------+---------+--------------------+--------------+--------------------+------+------------+--------+--------+--------+--------+--------------------+--------------------+-------+

In [7]:
df.printSchema()

print("Number of rows in the DataFrame:")
df.count()

root
 |-- DR_NO: integer (nullable = true)
 |-- Date Rptd: date (nullable = true)
 |-- DATE OCC: date (nullable = true)
 |-- TIME OCC: integer (nullable = true)
 |-- AREA : integer (nullable = true)
 |-- AREA NAME: string (nullable = true)
 |-- Rpt Dist No: integer (nullable = true)
 |-- Part 1-2: integer (nullable = true)
 |-- Crm Cd: integer (nullable = true)
 |-- Crm Cd Desc: string (nullable = true)
 |-- Mocodes: string (nullable = true)
 |-- Vict Age: integer (nullable = true)
 |-- Vict Sex: string (nullable = true)
 |-- Vict Descent: string (nullable = true)
 |-- Premis Cd: integer (nullable = true)
 |-- Premis Desc: string (nullable = true)
 |-- Weapon Used Cd: integer (nullable = true)
 |-- Weapon Desc: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Status Desc: string (nullable = true)
 |-- Crm Cd 1: integer (nullable = true)
 |-- Crm Cd 2: integer (nullable = true)
 |-- Crm Cd 3: integer (nullable = true)
 |-- Crm Cd 4: integer (nullable = true)
 |-- LOCA

2988445

In [8]:
df.show()

+---------+----------+----------+--------+-----+---------+-----------+--------+------+--------------------+--------------+--------+--------+------------+---------+--------------------+--------------+--------------------+------+------------+--------+--------+--------+--------+--------------------+--------------------+-------+---------+
|    DR_NO| Date Rptd|  DATE OCC|TIME OCC|AREA |AREA NAME|Rpt Dist No|Part 1-2|Crm Cd|         Crm Cd Desc|       Mocodes|Vict Age|Vict Sex|Vict Descent|Premis Cd|         Premis Desc|Weapon Used Cd|         Weapon Desc|Status| Status Desc|Crm Cd 1|Crm Cd 2|Crm Cd 3|Crm Cd 4|            LOCATION|        Cross Street|    LAT|      LON|
+---------+----------+----------+--------+-----+---------+-----------+--------+------+--------------------+--------------+--------+--------+------------+---------+--------------------+--------------+--------------------+------+------------+--------+--------+--------+--------+--------------------+--------------------+-------+

## Ζητούμενο 3

In [27]:
from pyspark.sql.functions import year, month, count, row_number
from pyspark.sql import Window

date_rptd = df.select('Date Rptd')
date_rptd = date_rptd.withColumn("Year", year("Date Rptd")).withColumn("Month", month("Date Rptd")).drop("Date Rptd")


crime_total = date_rptd.groupBy("Year", "Month").agg(count("*").alias("crime_total"))

# Define a window specification to partition by the "Year" column and order by the "crime_total" column
window_spec = Window().partitionBy("Year").orderBy(col("crime_total").desc())

# Use the row_number function to assign row numbers within each group
df_sorted = crime_total.withColumn("row_number", row_number().over(window_spec))

# Filter to keep only the top three within each group
df_top_three = df_sorted.filter(col("row_number") <= 3)

df_top_three.show(truncate=False)

# Save the DataFrame to a CSV file
df_top_three.write.csv("results/q1Dt.csv", header=True,  mode="overwrite")

import subprocess

hdfs_path = "hdfs://okeanos-master:54310/user/user/results/q1Dt.csv"
local_path = "/home/user/Project/results/"

subprocess.run(["hadoop", "fs", "-copyToLocal", hdfs_path, local_path])

+----+-----+-----------+----------+
|Year|Month|crime_total|row_number|
+----+-----+-----------+----------+
|2010|3    |17595      |1         |
|2010|7    |17520      |2         |
|2010|5    |17338      |3         |
|2011|8    |17139      |1         |
|2011|5    |17050      |2         |
|2011|3    |16951      |3         |
|2012|8    |17696      |1         |
|2012|10   |17477      |2         |
|2012|5    |17391      |3         |
|2013|8    |17329      |1         |
|2013|7    |16714      |2         |
|2013|5    |16671      |3         |
|2014|10   |12789      |1         |
|2014|7    |12696      |2         |
|2014|9    |12498      |3         |
|2015|8    |18951      |1         |
|2015|10   |18916      |2         |
|2015|7    |18528      |3         |
|2016|8    |19779      |1         |
|2016|10   |19615      |2         |
+----+-----+-----------+----------+
only showing top 20 rows



In [30]:
spark.stop()